#Bayesian optimization of hyperparameters for the QM8 dataset using Circular Fingerprints featurizer


In [0]:
#load deepchem
%%capture
%tensorflow_version 1.x
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -y -c deepchem -c rdkit -c conda-forge -c omnia deepchem-gpu=2.3.0
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [0]:
%%capture
%tensorflow_version 1.x
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [0]:
#copy local bug fix, until DeepChem update their repository
%cp gaussian_process.py /usr/local/lib/python3.7/site-packages/deepchem/hyper/gaussian_process.py

In [0]:
#copy local bug fix, until DeepChem update their repository
%cp transformers.py /usr/local/lib/python3.7/site-packages/deepchem/trans/transformers.py

In [4]:
import os
%env DEEPCHEM_DATA_DIR=/content

env: DEEPCHEM_DATA_DIR=/content


In [5]:
#deepchem modules
import deepchem as dc
from deepchem.models import GraphConvModel, DTNNModel
from deepchem.trans import undo_transforms

#tensorflow and keras modules
import tensorflow as tf
import keras
from tensorflow.keras.layers import Dense, Flatten, Input
from keras.callbacks import ModelCheckpoint


import numpy as np
import pandas as pd


import tempfile
from rdkit import Chem
from rdkit.Chem import Draw
from itertools import islice
from IPython.display import Image, display

%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf







/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Using TensorFlow backend.


In [6]:
!pip install pyGPGO


     |████████████████████████████████| 2.8 MB 4.1 MB/s 
     |████████████████████████████████| 908 kB 42.3 MB/s 
     |████████████████████████████████| 231 kB 53.7 MB/s 
     |████████████████████████████████| 1.5 MB 51.0 MB/s 
     |████████████████████████████████| 4.1 MB 47.5 MB/s 
     |████████████████████████████████| 668 kB 51.7 MB/s 
     |████████████████████████████████| 319 kB 58.3 MB/s 
  Created wheel for pyGPGO: filename=pyGPGO-0.4.0.dev1-py3-none-any.whl size=19865 sha256=8459ed46a960d69e1e78acbbb0ee223e33b3dbd92f97a16027dc0021e47a5b8a
  Stored in directory: /root/.cache/pip/wheels/52/6f/85/71782cff3fc9db678b2ac0d7c3fe44211b3ddeff2aabf4ea57
  Created wheel for theano: filename=Theano-1.0.4-py3-none-any.whl size=2667177 sha256=81234525a96f7db2f85cbd8c580139a3769d99cb973a011b03c9e935ce958043
  Stored in directory: /root/.cache/pip/wheels/33/e0/86/12647586a15bd29c062c9996231380908fb2dcf6a5df1c6f84
Successfully built pyGPGO theano


In [0]:
import pyGPGO

In [8]:
%cd /content/drive/My\ Drive/Colab\ Notebooks/DeepChem

/content/drive/My Drive/Colab Notebooks/DeepChem


In [0]:
#load unfeaturized dataset
%mkdir qm8
tasks, datasets, transformers = dc.molnet.load_qm8(featurizer='Raw',save_dir='qm8')

Loading raw samples now.
shard_size: 8192
Reading structures from /content/qm8.sdf.
Currently featurizing feature_type: RawFeaturizer
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
Featurizing sample 6000
Featurizing sample 7000
Featurizing sample 8000
Featurizing sample 9000
Featurizing sample 10000
Featurizing sample 11000
Featurizing sample 12000
Featurizing sample 13000
Featurizing sample 14000
Featurizing sample 15000
Featurizing sample 16000
Featurizing sample 17000
Featurizing sample 18000
Featurizing sample 19000
Featurizing sample 20000
Featurizing sample 21000
TIMING: featurizing shard 0 took 1.610 s
TIMING: dataset construction took 5.819 s
Loading dataset from disk.
TIMING: dataset construction took 1.395 s
Loading dataset from disk.
TIMING: dataset construction took 0.643 s
Loading dataset from disk.
TIMING: dataset construction took 0.646 s
Loading dataset from disk.
TIMING: data

In [0]:
train, valid, test = datasets

# Multitask

In [9]:
#load dataset from save_dif and apply featurizer to the train, valid and test sets

tasks, datasets, transformers = dc.molnet.load_qm8(featurizer='ECFP',save_dir='qm8')


Loading dataset from disk.
Loading dataset from disk.
Loading dataset from disk.


In [0]:
train, valid, test = datasets




# Hyperparameter optimization


In [0]:
from deepchem.hyper import GaussianProcessHyperparamOpt 

In [0]:
regression_metric = dc.metrics.Metric(dc.metrics.mean_absolute_error, mode="regression")
#regression_metric = dc.metrics.Metric(dc.metrics.mean_absolute_error)
#regression_metric = dc.metrics.Metric(dc.metrics.r2_score)
#len(regression_metric)
#dir(regression_metric)


In [19]:
optimizer = dc.hyper.GaussianProcessHyperparamOpt('tf_regression')
best_hyper_params, best_performance = optimizer.hyperparam_search(
  #  dc.molnet.preset_hyper_parameters.hps['tf_regression'],
    {'layer_sizes': [1000, 1000],
    'weight_init_stddevs': [0.02, 0.02],
     'bias_init_consts': [1., 1.],
     'dropouts': [0.25, 0.25],
     'penalty': 0.0005,
     'penalty_type': 'l2',
     'batch_size': 128,
     'nb_epoch': 10,
     'learning_rate': 0.0008        
    },
    train,
    valid,
    transformers,
    [regression_metric],
    n_tasks = 16,
    direction = False,
    max_iter = 50
)


Evaluation 	 Proposed point 	  Current eval. 	 Best eval.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
-----------------------------
Start fitting: tf_regression





computed_metrics: [0.03205359233066154, 0.02415784686603504, 0.04500566819197348, 0.0628470312597263, 0.037079102292562914, 0.029773392400936488, 0.043454148837559256, 0.050033233737682925, 0.03690596259236344, 0.028772998559827914, 0.04559385691943192, 0.04889793731610827, 0.03304605100526268, 0.027592128229072498, 0.05008246756734212, 0.05432252511530004]
computed_metrics: [0.03135949793336906, 0.023475467891738207, 0.04537335240478407, 0.06294559305685213, 0.036368771479530076, 0.029111847860069255, 0.0436478994902542, 0.049670655045317195, 0.036198147885203924, 0.028136562520409965, 0.04572836990098019, 0.0484982253635172, 0.032367144058331174, 0.026827702700884998, 0.05086510145499602, 0.05391790031352839]
-----------------------------
Start fitting: tf_regression
computed_metrics:

In [20]:
print(best_performance)

-0.35423250495919534


In [21]:
print(best_hyper_params)

{'layer_sizes': [328, 930], 'weight_init_stddevs': [0.04148276643011174, 0.042338157352892244], 'bias_init_consts': [0.8608654986713085, 2.1430632447723346], 'dropouts': [0.25, 0.25], 'penalty': 0.0012152145971319355, 'penalty_type': 'l2', 'batch_size': 324, 'nb_epoch': 10, 'learning_rate': 0.002398156662059796}


In [0]:
%cp /content/GPhypersearch.log /content/drive/My\ Drive/Colab\ Notebooks/DeepChem/.

# Run training with best parameters

In [12]:
#construct a multitask model
%rm -r qm8/optimized
%mkdir qm8/optimized
model_mc_ecfp = dc.models.MultitaskRegressor(
    n_tasks = len(tasks),
    n_features = train.X.shape[1],
    layers = [328, 930],
    weight_init_stddevs=[0.04148276643011174, 0.042338157352892244],
    bias_init_consts = [0.8608654986713085, 2.1430632447723346],
    dropouts = [0.25, 0.25],
    penalty = 0.0012152145971319355,
    penalty_type = 'l2',
    batch_size = 324,
    nb_epoch = 10,
    learning_rate = 0.002398156662059796,
    model_dir = 'qm8/optimized'
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
#callback function for checking validation loss on_epoch_end



 # def on_batch_end(self, batch, logs=None):
def on_batch_end(self, current_step):
       print('on end of batch',current_step)
       regression_metric = [dc.metrics.Metric(dc.metrics.mean_absolute_error, mode="regression")]
       train_scores = model_mc_ecfp.evaluate(train, regression_metric, transformers)
       train_scores['mean_absolute_error']
       train_scores_values = np.array(train_scores['mean_absolute_error'])
       print('Mean training MAE over all tasks: {:7.4f}'.format(np.sum(train_scores_values)))
       valid_scores = model_mc_ecfp.evaluate(valid, regression_metric, transformers)
       valid_scores_values = np.array(valid_scores['mean_absolute_error'])
       print('Mean validation MAE over all tasks: {:7.4f}'.format(np.sum(valid_scores_values)))


callbacks_list = [on_batch_end]


In [16]:
model_mc_ecfp.fit(train, nb_epoch=10, callbacks= callbacks_list)

computed_metrics: [0.01640215737508573, 0.015726547665205426, 0.02450596105071386, 0.042038791334767435, 0.01747764180721902, 0.016916333286181702, 0.023816199126138173, 0.03458243216338221, 0.017317258722396173, 0.017622537128361523, 0.02336967092321708, 0.034767362683050515, 0.0164964852428891, 0.015110447099185359, 0.026271051744264085, 0.03912863112763595]
Mean training MAE over all tasks:  0.3815
computed_metrics: [0.016730004636924856, 0.016023329584550168, 0.025062160018112616, 0.044505418819504955, 0.017799283858069698, 0.016896528667303543, 0.024113379891769104, 0.03623876096989595, 0.01795775339649949, 0.01789282731785848, 0.023633771865443555, 0.036310500621637265, 0.017008240348440735, 0.015417910762676007, 0.027617427251819926, 0.04105837828611477]
Mean validation MAE over all tasks:  0.3943
computed_metrics: [0.016223763433661217, 0.015000736932051905, 0.023539494599487187, 0.042065201652220004, 0.017248857154690777, 0.016180187792375036, 0.022936441790820515, 0.034524471

0.2606896816028489

In [0]:
#find and load best checking point
def find_best_checkpoint(model,model_dir,dataset,transformers):
  best_valid = float("inf") 
  metric = dc.metrics.Metric(dc.metrics.mean_absolute_error, mode="regression")
  for checkpoint_name in model.get_checkpoints(model_dir=model_dir):
    model.restore(checkpoint = checkpoint_name, model_dir = model_dir, session= None)
    valid_scores = model.evaluate(dataset, [metric], transformers)
    valid_scores_mean = np.sum(np.array(valid_scores['mean_absolute_error']))
    if valid_scores_mean < best_valid:
      best_checkpoint = checkpoint_name
    best_valid = min(best_valid,valid_scores_mean)
  print('found best checkpoint:',best_checkpoint,'with MAE sum=',np.sum(best_valid)) 
  model.restore(checkpoint = best_checkpoint, model_dir = model_dir, session= None)
  return 




  
  


In [28]:
#restore best checkpoint
find_best_checkpoint(model_mc_ecfp,'qm8/optimized',valid,transformers)

computed_metrics: [0.013561072759066197, 0.013305518817280851, 0.02105915167254526, 0.04084627629018933, 0.013531579342106543, 0.013253101631532064, 0.018692850354474252, 0.033386194463114936, 0.013730349268471683, 0.013195045471338906, 0.01855947424775844, 0.03363908282476849, 0.012806505715191194, 0.012293883540319399, 0.022602969751141656, 0.036547688313363184]
computed_metrics: [0.0124741725721073, 0.012421062208322401, 0.01967527315431832, 0.038300821247470654, 0.01250702802595717, 0.01217212810357688, 0.01799902072010601, 0.03229803413317953, 0.01259504683575535, 0.0121147614353995, 0.017743810844539895, 0.031739371566242076, 0.011786660138527215, 0.01146954224693538, 0.020400644090450313, 0.03413400812040159]
found best checkpoint: qm8/optimized/ckpt-2 with MAE sum= 0.3098313854432896


In [29]:
train_mae = model_mc_ecfp.evaluate(train, [dc.metrics.Metric(dc.metrics.mean_absolute_error, mode="regression")], transformers)
np.sum(train_mae['mean_absolute_error'])


computed_metrics: [0.009600048822973559, 0.00957815195806314, 0.01392234100475937, 0.027137739730238905, 0.009638828864573817, 0.009368875736585989, 0.012753128596597264, 0.021043709792822253, 0.009783494667080603, 0.00935081143854528, 0.01252415419218094, 0.020179578774874148, 0.009017267772216859, 0.008709493801359512, 0.013940293784733992, 0.023460862140001953]


0.22000878107760757

In [30]:
valid_mae = model_mc_ecfp.evaluate(valid, [dc.metrics.Metric(dc.metrics.mean_absolute_error, mode="regression")], transformers)
np.sum(valid_mae['mean_absolute_error'])

computed_metrics: [0.0124741725721073, 0.012421062208322401, 0.01967527315431832, 0.038300821247470654, 0.01250702802595717, 0.01217212810357688, 0.01799902072010601, 0.03229803413317953, 0.01259504683575535, 0.0121147614353995, 0.017743810844539895, 0.031739371566242076, 0.011786660138527215, 0.01146954224693538, 0.020400644090450313, 0.03413400812040159]


0.3098313854432896

In [31]:
test_mae = model_mc_ecfp.evaluate(test, [dc.metrics.Metric(dc.metrics.mean_absolute_error, mode="regression")], transformers)
np.sum(test_mae['mean_absolute_error'])

computed_metrics: [0.012602767229708241, 0.012483715195840434, 0.01972466604768546, 0.037967480231554385, 0.012720454501010567, 0.01273572195925083, 0.017849900466061493, 0.031016473806774386, 0.012961142750950954, 0.01265567232842864, 0.017661402205273697, 0.0304444365953993, 0.011996926591755015, 0.01175080981630546, 0.019901280930051876, 0.034345813016095086]


0.30881866367214583

In [0]:

def print_nice_table(tasks,test_mae):
  print('  task  ','   MAE   ')
  for task in tasks:
      print('  {0}  {1:7.4f}   '.format(task,test_mae['mean_absolute_error'][tasks.index(task)]))
  return

In [33]:
print_nice_table(tasks,test_mae)

  task      MAE   
  E1-CC2   0.0126   
  E2-CC2   0.0125   
  f1-CC2   0.0197   
  f2-CC2   0.0380   
  E1-PBE0   0.0127   
  E2-PBE0   0.0127   
  f1-PBE0   0.0178   
  f2-PBE0   0.0310   
  E1-PBE0   0.0127   
  E2-PBE0   0.0127   
  f1-PBE0   0.0178   
  f2-PBE0   0.0310   
  E1-CAM   0.0120   
  E2-CAM   0.0118   
  f1-CAM   0.0199   
  f2-CAM   0.0343   


In [26]:
#do 5 more epochs
model_mc_ecfp.fit(train, nb_epoch=5, callbacks= callbacks_list)


computed_metrics: [0.011674281049491586, 0.011485259472178294, 0.01737706095482649, 0.03419788238103976, 0.011558815369028298, 0.011621730107830163, 0.015235443601468586, 0.027556348576129697, 0.011908565478386129, 0.011563366731684893, 0.015061965851891373, 0.027581872519466373, 0.010913456455849844, 0.010709546294597541, 0.01805192300610052, 0.030116633718776778]
Mean training MAE over all tasks:  0.2766
computed_metrics: [0.013515956538055, 0.013199454051122819, 0.02104322286927951, 0.04089091201235692, 0.013518639457615212, 0.013095747268462472, 0.01873851724817954, 0.033364788830629724, 0.013669784529939106, 0.013089874579539402, 0.01856137578767874, 0.033620824598850775, 0.012742313504968558, 0.012230353852818503, 0.022515785580588574, 0.036511893144296904]
Mean validation MAE over all tasks:  0.3303
computed_metrics: [0.011616483700956973, 0.011422268992153196, 0.017325936012626195, 0.034142218009727575, 0.011637751879024957, 0.011837922043733037, 0.015370365748342812, 0.0274650

0.22288875976069408